In [ ]:
Both Batch Normalization (Batch Norm) and Layer Normalization (Layer Norm) are techniques used to stabilize and accelerate the training of deep neural networks by normalizing the inputs to each layer. Although they share a similar goal, they differ in what dimensions they normalize over and how they’re applied. Let’s break down each one in detail, including the mathematical formulation.

---

## Batch Normalization

### **Purpose and Intuition**

- **Internal Covariate Shift Reduction:**  
  Batch Norm was introduced to reduce the internal covariate shift—changes in the distribution of layer inputs during training—which can slow down learning.
  
- **Mini-Batch Dependency:**  
  The normalization is performed over a mini-batch of data for each feature independently.

### **Mathematical Formulation**

Consider a mini-batch of inputs for a particular feature:
\[
\{x^{(1)}, x^{(2)}, \dots, x^{(m)}\},
\]
where \( m \) is the batch size.

1. **Compute the Mean and Variance:**  
   \[
   \mu_B = \frac{1}{m} \sum_{i=1}^m x^{(i)}, \quad \sigma_B^2 = \frac{1}{m} \sum_{i=1}^m \left(x^{(i)} - \mu_B\right)^2.
   \]

2. **Normalize the Inputs:**  
   Each input is normalized using the computed mean and variance:
   \[
   \hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}},
   \]
   where \(\epsilon\) is a small constant to avoid division by zero.

3. **Scale and Shift:**  
   Batch Norm introduces learnable parameters \(\gamma\) (scale) and \(\beta\) (shift) to allow the network to recover the original distribution if needed:
   \[
   y^{(i)} = \gamma \hat{x}^{(i)} + \beta.
   \]

### **Key Characteristics**

- **Dependence on Batch Statistics:**  
  The normalization uses statistics (mean and variance) computed across the mini-batch.
- **Effective in CNNs:**  
  Works particularly well in convolutional networks where large mini-batches are available.
- **Potential Issues:**  
  In tasks like reinforcement learning or with very small mini-batches, the batch statistics might be noisy, impacting performance.

---

## Layer Normalization

### **Purpose and Intuition**

- **Normalization Across Features:**  
  Layer Norm is designed to normalize the inputs across the features for each individual data sample rather than across the batch.
  
- **Batch Independence:**  
  Since the normalization is computed per sample, it does not depend on the mini-batch size, making it especially useful in scenarios like recurrent neural networks (RNNs).

### **Mathematical Formulation**

Consider an input vector for a single data sample:
\[
\mathbf{x} = [x_1, x_2, \dots, x_n],
\]
where \( n \) is the number of features in that layer.

1. **Compute the Mean and Variance for the Sample:**  
   \[
   \mu_L = \frac{1}{n} \sum_{j=1}^n x_j, \quad \sigma_L^2 = \frac{1}{n} \sum_{j=1}^n \left(x_j - \mu_L\right)^2.
   \]

2. **Normalize the Features:**  
   Each feature is normalized using the sample statistics:
   \[
   \hat{x}_j = \frac{x_j - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}}.
   \]

3. **Scale and Shift:**  
   Like Batch Norm, learnable parameters \(\gamma_j\) and \(\beta_j\) are introduced for each feature:
   \[
   y_j = \gamma_j \hat{x}_j + \beta_j.
   \]

### **Key Characteristics**

- **Independence from Batch Size:**  
  Since normalization is applied per sample, Layer Norm works well even when the batch size is one.
- **Use in RNNs:**  
  Its independence from batch-level statistics makes it a natural fit for sequential models like RNNs, where the concept of a mini-batch is less straightforward.
- **Consistency Across Time Steps:**  
  For recurrent models, it helps maintain consistent normalization across different time steps in the sequence.

---

## Comparing Batch Norm and Layer Norm

| **Aspect**                   | **Batch Norm**                                         | **Layer Norm**                                        |
|------------------------------|--------------------------------------------------------|-------------------------------------------------------|
| **Normalization Dimension**  | Across the mini-batch for each feature                 | Across the features for each individual sample        |
| **Dependence on Batch Size** | Yes                                                    | No                                                    |
| **Best Suited For**          | Convolutional networks with large mini-batches         | Recurrent networks, transformer models, and small batch scenarios |
| **Computation of Statistics**| Uses mean and variance computed from the mini-batch    | Uses mean and variance computed from each sample      |

---

## Summary

- **Batch Normalization:**  
  Normalizes inputs over the mini-batch, which stabilizes and accelerates training in models where mini-batch statistics are robust (e.g., CNNs). It involves computing the mean and variance over the batch for each feature and then scaling and shifting the normalized values.

- **Layer Normalization:**  
  Normalizes inputs across the features within each individual sample, making it independent of the batch size. It is particularly useful in models like RNNs and transformers, where input distributions can vary significantly from one sample to another.

Both normalization techniques aim to reduce internal covariate shift and improve training dynamics, but the choice between them depends on the specific architecture and batch characteristics of your model.

Normalization placement in transformer-style architectures can significantly affect training stability and performance. Three common approaches are **Post-Norm**, **Pre-Norm**, and **Sandwich Norm**. Here’s a detailed explanation of each, including their mathematical formulation, pros, and cons.

---

## 1. Post-Norm

### **Configuration and Math**

In the **Post-Norm** setting, the layer normalization is applied **after** the sublayer (e.g., self-attention or feed-forward network) and its residual addition. A typical block is:

\[
y = \operatorname{LN}\Bigl(x + \operatorname{Sublayer}(x)\Bigr)
\]

- **Steps:**
  1. **Sublayer Transformation:** Compute \(\operatorname{Sublayer}(x)\).
  2. **Residual Addition:** Add the original input \(x\) to get \(x + \operatorname{Sublayer}(x)\).
  3. **Normalization:** Apply layer normalization (LN) to the sum.

### **Pros and Cons**

- **Pros:**
  - **Original Design:** This was used in the original Transformer paper (Vaswani et al., 2017) and works well for moderately deep models.
  - **Post-processing:** The normalization smooths the combined output before it is passed to the next block.

- **Cons:**
  - **Gradient Flow Issues:** In very deep networks, the gradients might become unstable because the normalization is applied after the residual addition. This can lead to vanishing or exploding gradients, making deep models harder to train.

---

## 2. Pre-Norm

### **Configuration and Math**

In the **Pre-Norm** configuration, the normalization is applied **before** the sublayer. The residual connection is added afterward:

\[
y = x + \operatorname{Sublayer}\Bigl(\operatorname{LN}(x)\Bigr)
\]

- **Steps:**
  1. **Normalization:** Apply LN to the input \(x\).
  2. **Sublayer Transformation:** Process the normalized input with \(\operatorname{Sublayer}(\operatorname{LN}(x))\).
  3. **Residual Addition:** Add the original \(x\) to the sublayer’s output.

### **Pros and Cons**

- **Pros:**
  - **Improved Gradient Flow:** Normalizing before the sublayer tends to stabilize training in very deep networks by ensuring the inputs to each sublayer have a consistent distribution.
  - **Deep Model Stability:** Pre-Norm architectures are more robust when stacking many layers, leading to more stable and efficient training.

- **Cons:**
  - **Slight Representational Shift:** Some researchers suggest that applying normalization before the sublayer might alter the learned representations compared to Post-Norm, potentially affecting the final performance on some tasks if the network isn’t very deep.

---

## 3. Sandwich Norm

### **Configuration and Math**

**Sandwich Norm** can be seen as a hybrid approach that “sandwiches” the sublayer between two normalization operations—one before and one after the transformation. A common formulation is:

\[
y = \operatorname{LN}\Bigl(x + \operatorname{Sublayer}\bigl(\operatorname{LN}(x)\bigr)\Bigr)
\]

- **Steps:**
  1. **Initial Normalization:** Apply LN to the input \(x\) to get \(\operatorname{LN}(x)\).
  2. **Sublayer Transformation:** Compute \(\operatorname{Sublayer}(\operatorname{LN}(x))\).
  3. **Residual Addition:** Add the original \(x\).
  4. **Final Normalization:** Apply LN to the result of the residual addition.

There are variants too; some may use additional or differently placed normalization steps. The key idea is to combine the benefits of both Pre-Norm and Post-Norm.

### **Pros and Cons**

- **Pros:**
  - **Enhanced Stability:** By normalizing both before and after the sublayer, Sandwich Norm can offer improved gradient flow and stable training, especially in very deep architectures.
  - **Balanced Representation:** The initial LN ensures stable inputs to the sublayer, while the final LN refines the output after adding the residual.

- **Cons:**
  - **Increased Computation:** Adding an extra normalization layer means additional computational overhead.
  - **Potential Redundancy:** In some cases, the extra normalization might be redundant, and careful hyperparameter tuning may be needed to ensure that it doesn’t adversely affect performance.

---

## Summary Comparison

| **Aspect**              | **Post-Norm**                                    | **Pre-Norm**                                  | **Sandwich Norm**                                                |
|-------------------------|--------------------------------------------------|-----------------------------------------------|------------------------------------------------------------------|
| **Order of Operations** | \(y = \operatorname{LN}(x + \operatorname{Sublayer}(x))\) | \(y = x + \operatorname{Sublayer}(\operatorname{LN}(x))\) | \(y = \operatorname{LN}(x + \operatorname{Sublayer}(\operatorname{LN}(x)))\) |
| **Gradient Flow**       | Can be unstable in very deep models              | Typically more stable in deep networks        | Enhanced stability by combining both strategies                  |
| **Computational Cost**  | Standard cost                                   | Standard cost                                 | Higher due to extra normalization step                           |
| **Best Use Cases**      | Moderate-depth networks                          | Very deep networks, RNNs, Transformers        | Deep architectures where extra normalization may help, but with tuning overhead |

---

## Final Thoughts

- **Post-Norm** is the original design and works well for shallower models but might suffer in very deep networks.
- **Pre-Norm** improves gradient flow and is generally favored for deep transformer models.
- **Sandwich Norm** aims to combine the best of both worlds by normalizing before and after the sublayer, potentially offering further stability improvements at the cost of increased computation and tuning complexity.

The choice among these normalization schemes depends on the specific architecture, model depth, and the particular training challenges you face.

Let’s dive into the details of three normalization techniques—Layer Norm, RMS Norm, and Deep Norm—by breaking down their calculations and the motivations behind each.

---

## 1. Layer Normalization

**Layer Normalization** normalizes the features within each individual sample. This makes it independent of the batch size and especially useful in settings like recurrent or transformer models.

### **Calculation Steps**

Given an input vector for one sample:
\[
\mathbf{x} = [x_1, x_2, \dots, x_n] \quad \in \mathbb{R}^n,
\]
we compute:

1. **Mean Calculation:**  
   Compute the mean of the features:
   \[
   \mu = \frac{1}{n} \sum_{i=1}^{n} x_i.
   \]

2. **Variance Calculation:**  
   Compute the variance (using all features):
   \[
   \sigma^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2.
   \]

3. **Normalization:**  
   Normalize each feature by subtracting the mean and dividing by the standard deviation (with a small \(\epsilon\) added for numerical stability):
   \[
   \hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}.
   \]

4. **Scale and Shift:**  
   Introduce learnable parameters \(\gamma\) (scale) and \(\beta\) (shift) to allow the model to recover the original distribution if needed:
   \[
   y_i = \gamma \, \hat{x}_i + \beta.
   \]

**Summary Equation:**
\[
\mathbf{y} = \gamma \odot \frac{\mathbf{x} - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta,
\]
where \(\odot\) denotes element-wise multiplication.

---

## 2. RMS Normalization (RMS Norm)

**RMS Norm** is similar to Layer Norm but simplifies the computation by **not subtracting the mean**. It only rescales the input based on its root-mean-square (RMS) value.

### **Calculation Steps**

Given the same input vector:
\[
\mathbf{x} = [x_1, x_2, \dots, x_n],
\]

1. **RMS Calculation:**  
   Compute the RMS value of the features:
   \[
   \text{RMS}(\mathbf{x}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2}.
   \]
   Notice that here we don’t subtract a mean—this makes RMS Norm invariant to the shift of the input.

2. **Normalization:**  
   Normalize the vector by dividing each element by the RMS value:
   \[
   \hat{x}_i = \frac{x_i}{\sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2 + \epsilon}}.
   \]

3. **Scale (and Optional Shift):**  
   Often, a learnable scaling parameter \(\gamma\) is applied:
   \[
   y_i = \gamma \, \hat{x}_i.
   \]
   In many implementations, no additive bias is used.

**Summary Equation:**
\[
\mathbf{y} = \gamma \odot \frac{\mathbf{x}}{\sqrt{\frac{1}{n}\sum_{i=1}^{n}x_i^2 + \epsilon}}.
\]

**Why Use RMS Norm?**  
- It’s computationally simpler (no mean computation) and may work well in settings where centering (subtracting the mean) is not critical.
- In some transformer-based models, it has been found to work as effectively as full layer normalization while reducing computation.

---

## 3. Deep Normalization (DeepNorm)

**DeepNorm** is a strategy developed to stabilize very deep networks—especially transformers—by carefully scaling residual connections. As networks get deeper, even small changes can accumulate and lead to instability. DeepNorm addresses this by introducing scaling factors that control the contribution of each sublayer’s output.

### **Key Idea and Motivation**

In standard residual blocks (with either Pre-Norm or Post-Norm), the output is typically computed as:
\[
x_{l+1} = \text{LN}\Bigl(x_l + F(x_l)\Bigr),
\]
where \(F(x_l)\) is the transformation (e.g., self-attention or feed-forward network) at layer \(l\).

When stacking many layers, the accumulation of these residuals can lead to exploding or vanishing activations. DeepNorm mitigates this by scaling the residual branch.

### **Calculation Steps and Formulation**

A common formulation of DeepNorm applies a scaling factor \(\alpha\) to the sublayer output:
\[
x_{l+1} = \text{LN}\Bigl(x_l + \alpha \cdot F(x_l)\Bigr).
\]

- **Choosing the Scaling Factor \(\alpha\):**  
  The scaling factor is typically chosen based on the total number of layers \(L\) in the network. A common heuristic is:
  \[
  \alpha = \frac{1}{\sqrt{L}},
  \]
  ensuring that the variance introduced by the residual branch remains controlled as the number of layers increases.

- **Optional Additional Scaling:**  
  In some variants, an extra learnable parameter (say \(\beta\)) might be introduced to further modulate the output:
  \[
  x_{l+1} = \beta \cdot \Bigl(x_l + \alpha \cdot F(x_l)\Bigr),
  \]
  where \(\beta\) can be learned during training.

### **Intuition Behind DeepNorm**

- **Stability in Deep Architectures:**  
  By scaling down the contribution of \(F(x_l)\), DeepNorm prevents the residuals from “overpowering” the input \(x_l\), thereby keeping the activations in a stable range.
  
- **Gradient Flow:**  
  With the residual branch scaled, the backpropagated gradients are less likely to explode or vanish, which is especially important when stacking hundreds of layers.

**Summary Equation (Simplest Form):**
\[
x_{l+1} = \text{LN}\Bigl(x_l + \frac{1}{\sqrt{L}} \, F(x_l)\Bigr).
\]

---

## Comparative Summary

| **Technique**  | **What It Normalizes**                               | **Key Formula**                                                                                                  | **Main Advantage**                                              |
|----------------|------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------|
| **Layer Norm** | All features per sample                              | \(\mathbf{y} = \gamma \odot \frac{\mathbf{x} - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta\)                         | Invariance to batch size; effective in RNNs/Transformers        |
| **RMS Norm**   | All features per sample (without centering)          | \(\mathbf{y} = \gamma \odot \frac{\mathbf{x}}{\sqrt{\frac{1}{n}\sum_{i=1}^{n}x_i^2 + \epsilon}}\)                   | Simpler computation; avoids mean subtraction effects            |
| **DeepNorm**   | Residual branches in deep networks                   | \(x_{l+1} = \text{LN}\Bigl(x_l + \frac{1}{\sqrt{L}} \, F(x_l)\Bigr)\) (with possible extra scaling \(\beta\))      | Stabilizes training of very deep networks by controlling residuals |

---

## Final Thoughts

- **Layer Norm** provides full normalization (centering and scaling) on a per-sample basis, making it a robust choice for many architectures.
- **RMS Norm** simplifies the process by focusing only on scaling via the RMS value, which can be computationally efficient while still providing normalization benefits.
- **DeepNorm** is less about normalizing the features within a single layer and more about ensuring that the contributions from deep residual connections do not destabilize training. By introducing a scaling factor (typically inversely related to the network depth), DeepNorm helps maintain a stable signal as it flows through many layers.

Each of these methods addresses different aspects of training stability and performance, and the choice among them depends on the network architecture, depth, and specific training challenges.